In [ ]:
select * from Szallashely for xml auto

In [ ]:
select sz.szoba_id, sz.szoba_szama, szh.szallas_nev
from Szallashely szh inner join Szoba sz on szh.szallas_id = sz.szallas_fk
for xml auto, elements

In [ ]:
select f.foglalas_pk as [@azon], v.nev as [@nev], f.mettol, f.felnott_szam
from Foglalas f inner join Vendeg v on f.ugyfel_fk = v.USERNEV
order by f.foglalas_pk ASC
for xml path('Foglalas'), root('Foglalasok')

In [ ]:
declare @xml XML
set @xml = cast('<Foglalasok>
	<Foglalas azon="558" nev="Fő Nándor">
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    <Foglalas azon="559" nev="Kelemen Áron">
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    </Foglalasok>' as xml)

select @xml.query('Foglalasok/Foglalas')

In [ ]:
declare @xml XML
set @xml = cast('<Foglalasok>
	<Foglalas azon="558">
    <Vendeg nev="Fő Nándor" />
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    <Foglalas azon="559">
    <Vendeg nev="Kelemen Áron" />
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    </Foglalasok>' as xml)

    select @xml.value('(/Foglalasok/Foglalas/Vendeg/@nev)[1]','nvarchar')

    select f.value('@foglalas_pk', 'int') as 'azonositok' 
    from @xml.nodes('/Foglalasok/Foglalas') as t(f)

In [ ]:
declare @idoc int
declare @xml XML
set @xml = cast('<Foglalasok>
	<Foglalas azon="558">
    <Vendeg nev="Fő Nándor" />
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    <Foglalas azon="559">
    <Vendeg nev="Kelemen Áron" />
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    </Foglalasok>' as xml)

    -- xml betoltes
    exec sp_xml_preparedocument @idoc OUTPUT, @xml

    select * from OPENXML(@idoc, 'Fogalalasok/Foglalas', 2)
    with(foglalas_pk int '@foglalas_pk', mettol date '@mettol', felnott_szam int '@felnott_szam')

    -- memoriafelszabaditas
    exec sp_xml_removedocument @idoc

In [ ]:
declare @i GEOGRAPHY
declare @j GEOGRAPHY

set @i = geography::STGeomFromText('POINT(-122.34900 47.65100)', 4326)
set @j = geography::STGeomFromText('POINT(-100.10 55.35)', 4326)

select @i.STDistance(@j)

In [ ]:
/*Készítsen függvényt UDFFoglalasszam néven, amely a paraméterként megadott ügyfél azonosítót felhasználva visszaadja, hogy az adott ügyfél eddig hányszor foglalt! 
	a) Teszteljük a fv. működését, pl: SELECT dbo.UDFFoglalasszam('kata')*/
   
CREATE OR ALTER FUNCTION UDFFoglalasszam(@ugyfel nvarchar(50))
RETURNS INT
AS
BEGIN
RETURN
(
  select count(1) 
  from Foglalas f INNER JOIN Vendeg v on f.ugyfel_fk = v.usernev 
  where v.usernev = @ugyfel
)
END

In [ ]:
-- test function
SELECT UDFFoglalasszam('kata')

In [ ]:
/*Készítsen triggert a Vendég táblához tgCheckUser néven, amely új vendég rögzítésekor ellenőrzi, hogy van-e már ilyen azonosítójú (USERNEV) a rendszerben. 
	a. Az új vendég adatait csak akkor rögzítsük, ha a megadott azonosító még nem létezik
	b. Ha már van ilyen, akkor adjunk róla hibaüzenetet!
	c.  Teszteljük a trigger működését a melléklet 1. pontjában lévő utasítással!*/

CREATE OR ALTER TRIGGER tgCheckUser ON Vendeg
INSTEAD OF INSERT
AS
BEGIN
DECLARE @vendeg nvarchar(50)
SELECT @vendeg = usernev
FROM inserted

IF @vendeg IN(SELECT usernev from Vendeg)
	BEGIN
    RAISERROR('Mar van ilyen user..',16,1)
	END
ELSE
	BEGIN
    INSERT INTO Vendeg SELECT * FROM inserted
    END
END

In [ ]:
-- test trigger
INSERT [dbo].[Vendeg] ([USERNEV], [NEV], [EMAIL], [SZAML_CIM], [SZUL_DAT]) VALUES (N'adam1', N'Kiss Ádám', N'ádám.kiss@mail.hu', N'5630 Békés Szolnoki út 8.', CAST(N'1991-12-28' AS Date))


In [2]:
declare @xml XML
set @xml = N'<CustomersOrders>
 <Customer custid="1">
 <!-- Comment 111 -->
 <companyname>Customer NRZBB</companyname>
 <Order orderid="10692">
 <orderdate>2007-10-03T00:00:00</orderdate>
 </Order>
 <Order orderid="10702">
 <orderdate>2007-10-13T00:00:00</orderdate>
 </Order>
 <Order orderid="10952">
 <orderdate>2008-03-16T00:00:00</orderdate>
 </Order>
 </Customer>
 <Customer custid="2">
 <!-- Comment 222 --> 
 <companyname>Customer MLTDN</companyname>
 <Order orderid="10308">
 <orderdate>2006-09-18T00:00:00</orderdate>
 </Order>
 <Order orderid="10952">
 <orderdate>2008-03-04T00:00:00</orderdate>
 </Order>
 </Customer>
</CustomersOrders>'

select @xml.query('/CustomersOrders/Customer/Order')

(1 row affected)

Total execution time: 00:00:00.052

(No column name)
"<Order orderid=""10692""><orderdate>2007-10-03T00:00:00</orderdate></Order><Order orderid=""10702""><orderdate>2007-10-13T00:00:00</orderdate></Order><Order orderid=""10952""><orderdate>2008-03-16T00:00:00</orderdate></Order><Order orderid=""10308""><orderdate>2006-09-18T00:00:00</orderdate></Order><Order orderid=""10952""><orderdate>2008-03-04T00:00:00</orderdate></Order>"
